In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 8.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.4 MB/s eta 0:00:00


In [ ]:
# 라이브러리
import warnings
warnings.filterwarnings('ignore')

import os
import time
import pickle
import random

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy
from torchsummary import summary

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
# OS 충돌 방지
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Matplotlib 커널 종료 방지
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# 시드 고정
def fixSEED(seed, deterministic=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

fixSEED(seed=42)

In [ ]:
# 디렉토리 설정
dir = 'Model'
path = '/content/drive/MyDrive/Projects/' + dir
print(path)
os.chdir(path)

/content/drive/MyDrive/Projects/Model


In [ ]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### 1. 데이터 불러오기 및 가공

In [ ]:
# Load data from a pickle file, and create labels
def callData(path: str):
    with open(path, "rb" ) as file:
        data = pickle.load(file)

    data['Label'] = np.where(data['useType'] == 0, 1, 0)

    return data

In [ ]:
# load data
data = callData("totalData.pkl")

In [ ]:
# check
data.head()

,partNum,newsID,newsCategory,newsSubcategory,newsContent,processLevel,processPattern,contentList,sentenceCount,processSentencenum,useType,wordCount,Label
0,P2,LC_M02_551626,생활&문화,NaN,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,하,99,"[손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다., 어떤 이에게는 ...",20,3,0,276,1
1,P2,LC_M02_551627,생활&문화,NaN,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이해...,하,99,[Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이...,16,3,0,204,1
2,P2,LC_M02_551629,생활&문화,NaN,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들여...,하,99,[요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들...,27,3,0,285,1
3,P2,LC_M02_551631,생활&문화,NaN,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,하,99,[생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가...,17,4,0,446,1
4,P2,LC_M02_551632,생활&문화,NaN,자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다. 직원들이 지역 사회...,하,99,"[자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다., 직원들이 지역 ...",16,2,0,196,1


In [ ]:
# Extract k sentences from the front and back of the data
def extract_sentences(row):
    # 비낚시성
    if row['Label'] == 0:
        front_sentences = row['contentList'][:3]
        back_sentences = row['contentList'][-3:]
    # 낚시성
    else:
        if row['processSentencenum'] == 1:
            front_sentences = row['contentList'][:2]
            back_sentences = row['contentList'][-2:]
        else:
            front_sentences = row['contentList'][:3]
            back_sentences = row['contentList'][-3:]

    return ' '.join(front_sentences), ' '.join(back_sentences)

# Apply extract_sentences to the data
def apply_extract_sentences(data):
    data['frontContent'], data['backContent'] = zip(*data.apply(extract_sentences, axis=1))

    return data

In [ ]:
# 앞, 뒤 n개의 문장 추출
data = apply_extract_sentences(data)

In [ ]:
# check
data.head()

,partNum,newsID,newsCategory,newsSubcategory,newsContent,processLevel,processPattern,contentList,sentenceCount,processSentencenum,useType,wordCount,Label,frontContent,backContent
0,P2,LC_M02_551626,생활&문화,NaN,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,하,99,"[손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다., 어떤 이에게는 ...",20,3,0,276,1,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,이 제품의 진정한 의미는 당신의 구매가 다른 사람에게 희망이 되고 살기 좋은 세상으...
1,P2,LC_M02_551627,생활&문화,NaN,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이해...,하,99,[Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이...,16,3,0,204,1,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이해...,그들은 라벨을 보거나 혹은 공급사에 직접 물어보는 방식을 통해서라도 이런 정보를 가...
2,P2,LC_M02_551629,생활&문화,NaN,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들여...,하,99,[요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들...,27,3,0,285,1,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들여...,우리 식품의약품안전처는 이런 흐름에 어떻게 반응할까. 우리 식료품 영양성분표시는 소...
3,P2,LC_M02_551631,생활&문화,NaN,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,하,99,[생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가...,17,4,0,446,1,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,예상대로진행된다면내년6월쯤국내첫월경컵이출시될것이다. 그러나 크라우드 펀딩의성공과관계...
4,P2,LC_M02_551632,생활&문화,NaN,자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다. 직원들이 지역 사회...,하,99,"[자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다., 직원들이 지역 ...",16,2,0,196,1,자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다. 직원들이 지역 사회...,안전 수칙은 비상사태가 발생했을 때 자원봉사자가 따라야 할 과정과 절차를 의미한다....


In [ ]:
# check
data['frontContent'][0]

'손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분과 지위를 상징하고 또 다른 이에게는 패션과 개성을 상징한다. 여기에 가난을 없애고 목마름을 해소시키며 암을 예방해준다면 그리고 가격이 40달러에 불과하다면 당신은 이 손목시계를 구매할 의향이 있는가'

### 2. Train/Valid/Test

In [ ]:
# Split into Train/Valid/Test
def splitData(data, SEED=42):
    train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=SEED)
    train_data, valid_data = train_test_split(train_val_data, test_size=0.25, random_state=SEED)

    print('학습용 데이터 개수 : {}개 \n검증용 데이터 개수 : {}개 \n평가용 데이터 개수 : {}개'.format(len(train_data), len(valid_data), len(test_data)))
    del train_val_data
    return train_data, valid_data, test_data

In [ ]:
train_data, valid_data, test_data = splitData(data)

학습용 데이터 개수 : 157390개 
검증용 데이터 개수 : 52464개 
평가용 데이터 개수 : 52464개


### 3.  Dataset, Dataloader 정의

In [ ]:
# 파라미터 개수 확인
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable Params: {trainable_params} \nAll params: {all_param}")
    print('-'*50)
    print(f"Trainable(%): {round(100 * trainable_params / all_param, 5)}")

In [ ]:
# One-way BERT Dataset
class OwoWayBERTDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['newsContent']
        label = self.data.iloc[idx]['Label']

        inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_length)

        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        return input_ids, attention_mask, torch.tensor(label, dtype=torch.long) # 정수형

### 4. 모델 정의

In [ ]:
class OneWayBERT(nn.Module):
    def __init__(self, model_checkpoint, lora_r=4, lora_alpha=1, lora_dropout=0.1):
        super(OneWayBERT, self).__init__()

        # Pre-trained Model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

        # LoRA Config
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            target_modules=["query", "value"],  # Apply LoRA to the attention layers
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout
        )

        # Apply LoRA to Pre-trained Layers
        self.model = get_peft_model(self.model, lora_config)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask).logits
        return outputs

### 5. 학습, 검증 진행 함수

In [ ]:
class Trainer:
    def __init__(self, model, train_dataloader, valid_dataloader, criterion, optimizer, acc_metric, f1_metric, device, num_epochs, wait_for_es):
        self.model = model  # model for training
        self.train_dataloader = train_dataloader  # training dataloader
        self.valid_dataloader = valid_dataloader  # validation dataloader
        self.criterion = criterion  # loss function
        self.optimizer = optimizer  # optimizer
        self.acc_metric = acc_metric  # accuracy metric
        self.f1_metric = f1_metric  # f1 score metric
        self.device = device  # device(cuda or cpu)
        self.num_epochs = num_epochs  # number of epochs
        self.wait_for_es = wait_for_es  # early stopping patience

    def train(self):
        train_loss_history, train_acc_history, train_f1_history = [], [], []
        valid_loss_history, valid_acc_history, valid_f1_history = [], [], []

        best_loss = float('inf')
        count = 0
        best_acc = 0
        start_time = time.time()

        for epoch in range(self.num_epochs):
            print('-' * 80)
            print(f'Epoch {epoch+1}/{self.num_epochs}')
            print('-' * 80)

            train_loss, train_acc, train_f1 = self._train_model()
            valid_loss, valid_acc, valid_f1 = self._valid_model()

            train_loss_history.append(train_loss)
            train_acc_history.append(train_acc)
            train_f1_history.append(train_f1)

            valid_loss_history.append(valid_loss)
            valid_acc_history.append(valid_acc)
            valid_f1_history.append(valid_f1)

            elapsed_time = time.time() - start_time
            print(f'[Train] \t Loss: {train_loss:.4f} \t Acc: {train_acc:.4f} \t F1: {train_f1:.4f}')
            print(f'[Valid] \t Loss: {valid_loss:.4f} \t Acc: {valid_acc:.4f} \t F1: {valid_f1:.4f}')
            print(f'(Epoch {epoch+1} complete in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s)')

            if valid_acc > best_acc:
                best_acc = valid_acc
                try:
                    torch.save(self.model.state_dict(), f'./RoBERTa_ACC_{valid_acc:.4f}.pth')
                except:
                    print('Model Not Saved')
                    continue

            if valid_loss < best_loss:
                best_loss = valid_loss
                count = 0
            else:
                count += 1
                if count >= self.wait_for_es:
                    print(f'Early Stopping at Epoch {epoch+1}')
                    break

        return train_loss_history, train_acc_history, train_f1_history, valid_loss_history, valid_acc_history, valid_f1_history

    # Training function
    def _train_model(self):
        self.model.train()

        train_loss = 0
        train_acc = 0
        train_f1 = 0

        for batch in tqdm(self.train_dataloader, desc='Training'):
            input_ids, attention_mask, y = batch
            input_ids, attention_mask, y = input_ids.to(self.device), attention_mask.to(self.device), y.to(self.device)

            outputs = self.model(input_ids, attention_mask)
            loss = self.criterion(outputs, y) # 데이터셋에서 레이블을 long으로 정의함

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            preds = torch.argmax(outputs, dim=1)
            train_loss += loss.item()
            train_acc += self.acc_metric(preds, y).item()
            train_f1 += self.f1_metric(preds, y).item()

        return train_loss / len(self.train_dataloader), train_acc / len(self.train_dataloader), train_f1 / len(self.train_dataloader)

    # Validating function
    def _valid_model(self):
        self.model.eval()

        valid_loss = 0
        valid_acc = 0
        valid_f1 = 0

        with torch.no_grad():
            for batch in tqdm(self.valid_dataloader, desc='Validating'):
                input_ids, attention_mask, y = batch
                input_ids, attention_mask, y = input_ids.to(self.device), attention_mask.to(self.device), y.to(self.device)

                outputs = self.model(input_ids, attention_mask)
                loss = self.criterion(outputs, y.long())

                preds = torch.argmax(outputs, dim=1)
                valid_loss += loss.item()
                valid_acc += self.acc_metric(preds, y).item()
                valid_f1 += self.f1_metric(preds, y).item()

        return valid_loss / len(self.valid_dataloader), valid_acc / len(self.valid_dataloader), valid_f1 / len(self.valid_dataloader)


### 6. 학습 진행

In [ ]:
# 데이터셋 파라미터
batch_size = 32
num_labels = 2
max_length = 64

model_checkpoint = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
# 데이터셋 정의
train_dataset = OwoWayBERTDataset(train_data, tokenizer)
valid_dataset = OwoWayBERTDataset(valid_data, tokenizer)
test_dataset = OwoWayBERTDataset(test_data, tokenizer)

In [ ]:
# 데이터로더 정의
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

next(iter(train_dataloader))

[tensor([[    0, 15093,  2440,  ...,  7635,  2470,     2],
         [    0,  3997,  2720,  ...,  2259,  3699,     2],
         [    0,  7325,  2052,  ...,  1873,  2088,     2],
         ...,
         [    0,  3735,  2079,  ...,  2259,  3814,     2],
         [    0, 10979,  2218,  ...,  1037,  2279,     2],
         [    0, 28712,  2052,  ...,  2170,  4576,     2]]),
 tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 tensor([1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
         1, 0, 1, 0, 1, 0, 1, 0])]

In [ ]:
# Model
model = OneWayBERT(model_checkpoint)
model.to(device)

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OneWayBERT(
  (model): PeftModelForSequenceClassification(
    (base_model): LoraModel(
      (model): RobertaForSequenceClassification(
        (roberta): RobertaModel(
          (embeddings): RobertaEmbeddings(
            (word_embeddings): Embedding(32000, 1024, padding_idx=1)
            (position_embeddings): Embedding(514, 1024, padding_idx=1)
            (token_type_embeddings): Embedding(1, 1024)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): RobertaEncoder(
            (layer): ModuleList(
              (0-23): 24 x RobertaLayer(
                (attention): RobertaAttention(
                  (self): RobertaSelfAttention(
                    (query): lora.Linear(
                      (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.1, inplace

In [ ]:
# LoRA 적용 후 훈련 가능한 파라미터 개수
print_trainable_parameters(model)

Trainable Params: 1444866 
All params: 338103300
--------------------------------------------------
Trainable(%): 0.42734


In [ ]:
# 손실함수와 옵티마이저, 그리고 Metrics
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
acc_metric = BinaryAccuracy().to(device)
f1_metric = BinaryF1Score().to(device)

In [ ]:
# 모델 학습 파라미터
num_epochs = 3
wait_for_es = 10

In [ ]:
# Trainer 정의
trainer = Trainer(model,
                  train_dataloader,
                  valid_dataloader,
                  criterion,
                  optimizer,
                  acc_metric,
                  f1_metric,
                  device,
                  num_epochs,
                  wait_for_es)

In [ ]:
# 학습 시작
train_loss_history, train_acc_history, train_f1_history, valid_loss_history, valid_acc_history, valid_f1_history = trainer.train()

--------------------------------------------------------------------------------
Epoch 1/3
--------------------------------------------------------------------------------


Validating: 100%|██████████| 1640/1640 [07:05<00:00,  3.86it/s]


[Train] 	 Loss: 0.6810 	 Acc: 0.5552 	 F1: 0.5282
[Valid] 	 Loss: 0.6652 	 Acc: 0.5874 	 F1: 0.5662
(Epoch 1 complete in 45m 17s)
--------------------------------------------------------------------------------
Epoch 2/3
--------------------------------------------------------------------------------


Validating: 100%|██████████| 1640/1640 [07:06<00:00,  3.84it/s]


[Train] 	 Loss: 0.6641 	 Acc: 0.5906 	 F1: 0.5764
[Valid] 	 Loss: 0.6516 	 Acc: 0.6110 	 F1: 0.6051
(Epoch 2 complete in 91m 9s)
--------------------------------------------------------------------------------
Epoch 3/3
--------------------------------------------------------------------------------


Validating: 100%|██████████| 1640/1640 [07:06<00:00,  3.84it/s]


[Train] 	 Loss: 0.6540 	 Acc: 0.6067 	 F1: 0.5981
[Valid] 	 Loss: 0.6424 	 Acc: 0.6213 	 F1: 0.6319
(Epoch 3 complete in 137m 0s)
